In [4]:
import tensorflow as tf
import time
import numpy as np
import os
from tensorflow import keras
import pprint

In [5]:
use_tpu = True #@param {type:"boolean"}

if use_tpu:
    assert 'COLAB_TPU_ADDR' in os.environ, 'Missing TPU; did you request a TPU in Notebook Settings?'

if 'COLAB_TPU_ADDR' in os.environ:
  TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
else:
  TF_MASTER=''

with tf.compat.v1.Session(TF_MASTER) as session:
  print ('List of devices:')
  pprint.pprint(session.list_devices())

List of devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 881440018777266711),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 4312646848002408110),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 4567475986391415858),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, -2259003416087525562),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 6218320445764940763),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 4044634075013356987),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 6804726289300748360),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, -7541924663223483815),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 4536375203606212004),
 _DeviceAttributes(/job:tpu_w

In [6]:
def movable_condition(first, second):
    """Define whether two close tile can be merged"""
    return ((first == 0) and (second != 0)) or \
           ((np.any(np.array([first, second]) > 2)) and (first == second)) or \
           ((first, second) == (1, 2)) or \
           ((second, first) == (1, 2))


def can_move_col(array):
    """Check whether an array can be merged."""
    for i in range(3):
        first, second = array[i], array[i + 1]
        if movable_condition(first, second):
            return True
    return False


def allowed_moves(state):
    """Find allowed moves for a certain game state"""
    allowed_actions = []
    # Check whether the agent can swipe up
    if np.any([can_move_col(col) for col in state.T]):
        allowed_actions.append('w')
    # Check whether the agent can swipe down
    if np.any([can_move_col(col[::-1]) for col in state.T]):
        allowed_actions.append('s')
    # Check whether the agent can swipe left
    if np.any([can_move_col(row) for row in state]):
        allowed_actions.append('a')
    # Check whether the agent can swipe right
    if np.any([can_move_col(row[::-1]) for row in state]):
        allowed_actions.append('d')
    return allowed_actions


def try_move_col(array):
    """Return the next state for an array"""
    new_array = array.copy()
    for i in range(3):
        first, second = array[i], array[i + 1]
        if movable_condition(first, second):
            new_array[i] = first + second
            new_array[i + 1:] = np.append(new_array[i + 2:], 0)
            return new_array
        else:
            continue


def get_reward(current_state, next_state):
    """Given the current state and the next state, return the reward for the transition action."""
    reward = 0
    # maximum number gets larger
    reward += (np.max(next_state) - np.max(current_state))
    # more merge
    reward += (get_score(next_state) - get_score(current_state))
    return reward


def get_score(state):
    """Get score given a game state"""
    all_power = [3**(np.log2(num/3)+1) for row in state for num in row if num > 3]
    return np.sum(all_power)


def try_move(current_state, action):
    """Given the state and the chosen action, return the next state"""
    next_state = current_state.copy()
    allowed_actions = allowed_moves(current_state)
    if action not in allowed_actions:
        print(f'Can not move {action}')
        return current_state

    # Swipe up
    if action == 'w':
        for i, col in enumerate(current_state.T):
            if can_move_col(col):
                next_state.T[i] = try_move_col(col)
    # Swipe down
    elif action == 's':
        for i, col in enumerate(current_state.T):
            if can_move_col(col[::-1]):
                new_array = try_move_col(col[::-1])
                next_state.T[i] = new_array[::-1]
    # Swipe left
    elif action == 'a':
        for i, col in enumerate(current_state):
            if can_move_col(col):
                next_state[i] = try_move_col(col)
    # Swipe right
    elif action == 'd':
        for i, col in enumerate(current_state):
            if can_move_col(col[::-1]):
                new_array = try_move_col(col[::-1])
                next_state[i] = new_array[::-1]

    elif action == 'stop':
        return current_state, get_score(current_state)

    reward = get_reward(current_state, next_state)

    return next_state, reward


def hashable(state):
    """Switch state matrix to string matrix, so as to make it hashable."""
    return ', '.join([str(int(i)) for row in state for i in row])


def _select_best_move(game):
    """Selects best move that can get the maximum reward for the next state"""
    possible_next_actions = allowed_moves(game.state)
    state_action_score = [(move, try_move(game.state, move)[1])
                          for move in possible_next_actions]
    max_score = max(state_action_score, key=lambda item: item[1])[1]
    max_move_list = [move for move, score in state_action_score if score == max_score]
    best_next_move = np.random.choice(max_move_list)
    return best_next_move

In [7]:
class Threes:
    """
    This is a simulated environment of game Threes.
    Swipe direction: {left: 'a', right: 'd', up: 'w', down: 's'}.
    There are two levels for this game: ['hard', 'easy'], in which default level is 'hard'.
    """

    def __init__(self, level='hard'):
        """Initialize the game"""
        self.state = np.zeros((4, 4))
        x, y = np.random.choice(4, 2)
        self.state[x, y] = np.random.choice([1, 2])
        self.score = get_score(self.state)
        self.level = level

    def playable(self):
        """Check whether the game is still playable."""
        if len(allowed_moves(self.state)) != 0:
            return True
        else:
            return False

    def gen_new_tile(self):
        """Generate a new tile after each move."""

        # Basic list of numbers that can be selected
        choice_list = [1, 2, 3]

        # More number can be selected when the maximum number on the grid gets larger
        if np.max(self.state) % 3 == 0:
            max_power = np.int(np.log2(np.max(self.state) / 3))
            choice_list += [3 * 2 ** i for i in range(max_power + 1)]

        # Generate the probabilities for each candidate
        if self.level == 'hard':
            norm_prob = [1 / len(choice_list)] * len(choice_list)
        else:
            prob = [i + 1 for i in range(len(choice_list))][::-1]
            norm_prob = [num / sum(prob) for num in prob]

            # return next number
        return np.random.choice(choice_list, p=norm_prob)

    def make_move(self, action):
        """Given the action, the game goes to the next state"""
        if action == 'stop':
            self.score = get_score(self.state)
            return self.state, self.score

        self.state = try_move(self.state, action)[0]

        # generate new tile for the current state
        new_tile = self.gen_new_tile()
        loc_0 = np.argwhere(self.state == 0)
        x, y = loc_0[np.random.choice(len(loc_0))]

        # Update the game state and scores
        self.state[x, y] = new_tile
        self.score = get_score(self.state)

In [8]:
class Agent:
    """
    This is an agent to play game "Threes". There are two main mode to play the game. One is human mode and the other
    is computer mode(demo game). For the computer mode, there are currently three methods to play the game: [
    'random', 'max', 'q-learning'] The functions here are inspired by
    "https://github.com/brianspiering/rl-course/blob/master/labs/lab_4_tic_tac_toe/lab_4_tic_tac_toe.ipynb"
    """

    def __init__(self, threes, epsilon=0.1, alpha=1.0):
        """Initial the Agent."""
        self.V = dict()
        self.NewGame = threes
        self.epsilon = epsilon
        self.alpha = alpha

    def state_value(self, game_state, action):
        """Look up state value. If never seen state, then assume 0."""
        return self.V.get((hashable(game_state), action), 0.0)

    def state_values(self, game_state, actions):
        """Return a dictionary of state-value pair. It is for finding the action that can maximize the q value """
        return dict(((hashable(game_state), action), self.state_value(game_state, action)) for action in actions)

    def learn_game(self, n_episodes=1000):
        """Let's learn through complete experience to get that reward."""
        for e in range(1, n_episodes + 1):
            game = self.NewGame()
            while game.playable():
                action, reward = self.learn_from_move(game)
            self.V[(hashable(game.state), action)] = reward

    def learn_from_move(self, game):
        """The heart of Q-learning."""

        current_state = game.state
        # Select next action with epsilon-greedy method
        selected_move = self.learn_select_move(game)

        # Next state s(t+1) and reward r
        next_state, reward = try_move(current_state, selected_move)

        # Current state Q value Q(s, a)
        old_value = self.state_value(current_state, selected_move)

        # best action a* for the next state with the largest q value Q(st+1, a*)
        next_max_V, next_max_move = self.select_best_move(game, next_state)

        # Q-learning that updates the q-value
        self.V[(hashable(current_state), selected_move)] = (1 - self.alpha) * old_value + self.alpha * (
                    reward + next_max_V)

        game.make_move(selected_move)
        return selected_move, reward

    def learn_select_move(self, game):
        """Exploration and exploitation"""
        if np.random.uniform(0, 1) < self.epsilon:
            selected_action = np.random.choice(allowed_moves(game.state))
        else:
            selected_action = self.select_best_move(game, game.state)[1]
        return selected_action

    def select_best_move(self, game, game_state):
        """Selects best move for given state(Greedy)"""
        state_action_values = self.state_values(game_state, allowed_moves(game_state))
        max_V = max(state_action_values.values())
        max_move = np.random.choice([state_action[1] for state_action, v in state_action_values.items() if v == max_V])
        return max_V, max_move

    def demo_game(self, level='hard', mode='random'):
        """Agent plays with different policies (random/max/q-learning)"""
        game = self.NewGame()
        game.level = level
        while game.playable():
            if mode == 'random':
                next_action = np.random.choice(allowed_moves(game.state))
            elif mode == 'max':
                next_action = _select_best_move(game)
            elif mode == 'q-learning':
                next_action = self.select_best_move(game, game.state)[1]
            else:
                return "No such mode"
            game.make_move(next_action)
        return game.score

    def human_mode(self):
        """Interactive mode"""
        game = self.NewGame()
        level = input('level: easy or hard?')
        game.level = level
        print(game.state)
        while game.playable():
            human_allowed_moves = allowed_moves(game.state) + ['stop']
            human_move = input(f'You can input {human_allowed_moves}')
            if human_move == 'stop':
                return f'Game over! Your score is {game.score}'
            game.make_move(human_move)
            print(game.state)
        return f'Game over! Your score is {game.score}'

In [11]:
def print_demo_game_stats(agent, n_games=100, level='Hard', mode='random'):
    """print the result(mean score and max score) of playing demo game for some times"""
    results = [agent.demo_game(level=level, mode=mode) for _ in range(n_games)]
    mean_score, max_score = np.mean(results), np.max(results)
    print(f"mean score: {mean_score}")
    print(f"max score: {max_score}")
    return mean_score, max_score


def train_qlearning(agent, n_games=100, n_episodes=100, n_training_blocks=10, level='Hard'):
    """Given agent, do more training. Return (hopefully) improved agent."""
    for n_training_block in range(1, n_training_blocks + 1):
        agent.learn_game(n_episodes)
        print(f"After {n_episodes * n_training_block:,} learning games:")
        mean_score, max_score = print_demo_game_stats(agent, n_games=n_games, level=level, mode='q-learning')
    return agent, mean_score, max_score

In [12]:
# Without learning, machine plays by choosing the move that maximize the score of next state.
agent = Agent(Threes)

In [13]:
print("No training:\n Random Mode:")
mean_score_random, max_score_random = print_demo_game_stats(agent, n_games=100, level='Hard', mode='random')
print('Greedy Mode:')
mean_score_max, max_score_max = print_demo_game_stats(agent, n_games=100, level='Hard', mode='max')


No training:
 Random Mode:
mean score: 327.6
max score: 2502.0
Greedy Mode:
mean score: 446.76
max score: 4122.0


In [14]:
import logging
logging.getLogger('tensorflow').disabled = True

In [15]:
class ExperienceReplay():
    "Store the agent's experiences inorder to collect enough example to get a reward signal."
    
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, states, game_over):
        self.memory.append([states, game_over])
        
        # If memory is too large, then evict to reduce memory size
        if len(self.memory) > self.max_memory:
            # Evict oldest
            del self.memory[0]
            
    def get_batch(self, model, batch_size=10):
        len_memory = len(self.memory)
        # Dimension of the output layer of NN
        n_actions = model.layers[1].output_shape[1]
        # Dimension of the input layer of NN
        env_dim = model.layers[0].input_shape[1]
        
        inputs = np.zeros((min(len_memory, batch_size), env_dim))
        targets = np.zeros((inputs.shape[0], n_actions))
        
        for i, idx in enumerate(np.random.randint(0, len_memory, size=inputs.shape[0])):
            state_t, action_t, reward_t, state_tp1 = self.memory[idx][0]
            game_continue = self.memory[idx][1]
            # train_x: st 
            inputs[i:i+1] = state_t
            # Initialize target y: q values Q(st, a0), Q(st, a1), Q(st, a2) for each action
            targets[i] = model.predict(state_t)[0]
            # Next max q value for the next state: maxQ(stp1, a*)
            q_sa = np.max(model.predict(state_tp1))
            if game_continue:
                # Update Q(st,at) with Q-learning 
                targets[i, action_t] = reward_t + q_sa
            else:
                targets[i, action_t] = reward_t
        return inputs, targets

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer

model = Sequential()

# Input layer
model.add(InputLayer(input_shape=(16,)))

# The second layer (hidden)
model.add(Dense(units=32, activation='relu')) 

# # Second hidden layer
# model.add(Dense(units=64,activation='relu')) 

# # Third hidden layer
# model.add(Dense(units=64,activation='relu')) 

# Output layer
model.add(Dense(units=4,activation='softmax')) 


model.compile(optimizer='adam', loss="categorical_crossentropy")

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                544       
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 132       
Total params: 676
Trainable params: 676
Non-trainable params: 0
_________________________________________________________________


In [17]:
def hash_num(state):
    return np.array([[int(i) for row in state for i in row]])

In [ ]:
# Run Training

# Define environment
action_map = {0: 'a', 1: 'w', 2:'s', 3:'d'}
key_list = list(action_map.keys())
val_list = list(action_map.values())

# Initialize experience replay object
exp_replay = ExperienceReplay(max_memory=2048)

# Exploration rate
epsilon = .1  

# Training variables
n_episodes = 11
results = []
loss = float('inf')
    
for e in range(1, n_episodes): 

    if (e == 1) or (e % 10 == 1) :
        print(f"Epoch: {e:03d}/{n_episodes:,} | Loss value: {loss:>6.3f} | Mean score: {np.mean(results)} | Max score: {np.max(results)}")
        
    # The next new episode.
    game_threes = Threes()
 
    # While the game is not over
    while game_threes.playable():
        
        # Get initial input (as vector).
        current_state = game_threes.state
        allowed_actions =  allowed_moves(current_state)
        
        # Get next action - You guessed it eplison-greedy.
        if np.random.rand() > epsilon:
            q = model.predict(hash_num(current_state))
            action = action_map[np.argmax(q[0])]
        else:
            action = np.random.choice(allowed_actions)
        if action not in allowed_actions:
            action = np.random.choice(allowed_actions)
        
        # Apply action, get rewards and new state.
        next_state, reward = try_move(current_state, action)
        game_threes.make_move(action)
        action_num = key_list[val_list.index(action)]
        
        # Store experience.
        exp_replay.remember([hash_num(current_state), action_num, reward, hash_num(next_state)], game_threes.playable())

        # Get collected data to train model.
        inputs, targets = exp_replay.get_batch(model, batch_size=50)

        # Train model on experiences.
        loss = model.train_on_batch(inputs, targets)
        
    results.append(get_score(game_threes.state))


Epoch: 001/1,001 | Loss value: 32.135 | Mean score: 207.0 | Max score: 207.0
Epoch: 002/1,001 | Loss value: 11.065 | Mean score: 162.0 | Max score: 207.0
Epoch: 003/1,001 | Loss value: 10.442 | Mean score: 174.0 | Max score: 207.0
Epoch: 004/1,001 | Loss value:  7.267 | Mean score: 218.25 | Max score: 351.0
Epoch: 005/1,001 | Loss value: 44.590 | Mean score: 210.6 | Max score: 351.0
Epoch: 006/1,001 | Loss value: 56.962 | Mean score: 310.5 | Max score: 810.0
Epoch: 007/1,001 | Loss value: 45.314 | Mean score: 389.57142857142856 | Max score: 864.0
Epoch: 008/1,001 | Loss value: 110.463 | Mean score: 380.25 | Max score: 864.0
Epoch: 009/1,001 | Loss value: 356.574 | Mean score: 349.0 | Max score: 864.0
Epoch: 010/1,001 | Loss value: 93.510 | Mean score: 383.4 | Max score: 864.0
Epoch: 011/1,001 | Loss value: 722.636 | Mean score: 373.90909090909093 | Max score: 864.0
Epoch: 012/1,001 | Loss value: 82.391 | Mean score: 355.5 | Max score: 864.0
Epoch: 013/1,001 | Loss value: 1601.579 | Mea